# Lesson 4 Continued: Assessing Models

Today:
1. Assessing Models
    + Training and Test data
    + Assessing the goodness of a regression model
    + Overfitting, Underfitting

## 1. Training and Test data

So far:

**Step 1:** Find patterns in data
(e.g., there is a linear relationship between x and y)

**Step 2:** Build a model that fits the data relatively well  (e.g., fit a line through the plotted data points)

**Step 3:** Assess the model
- *Qualitatively:* What are the limitations of the model?
- *Quantitatively:* If there are **new x values**, how good would the model  be in predicting the y values?


**In order to assess the model:**

- We need two sets of data
    - First set: Used only to find  patterns and build model
    - Second set: Used only to assess  the model


<img src='train_test_split.png' width=800>

### Example

Our goals:
- Split the dataset into training and test data
- Create a model using the training data
- Assess the model using the test data
    - Compare actual vs. predicted
    - How large are the errors?


In [ ]:
# load libraries
import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
# Read in data
top50 = pd.read_csv('../../../shared/datasets/top50.csv')

top50.head()

In [ ]:
top50.shape

**Training and Test Data Method 1:** One way to split the data into training and test data is to take the first half as training and the second half and test data.

In [ ]:
# split into test and training
#   training_data = row 0 to row 24
#   test_data = row 25 to row 49

training_data = top50.iloc[ 0:25 , : ]
test_data = top50.iloc[ 25:50 , :]

In [ ]:
training_data.corr(numeric_only=True)

In [ ]:
# plot
sns.regplot(  data = training_data , x = 'LoudnessdB', y = 'Energy'   )

In [ ]:
from sklearn.linear_model import LinearRegression

X_train = training_data[['LoudnessdB']]
y_train = training_data['Energy']


lin_model = LinearRegression().fit( X_train, y_train  )

m = lin_model.coef_
b = lin_model.intercept_

print(m)
print(b)
print(f"Equation of best linear curve is y = {round(m[0], 3)}x + {round(b, 3)}")
# predicted energy = m * loudnessdb + b

In [ ]:
# predict y values for the test dataset

test_data.head()

test_data['Energy_predicted'] = test_data['LoudnessdB'] * m + b

# compute prediction error
test_data['Error'] = test_data['Energy'] - test_data['Energy_predicted']
test_data['Error_squared'] = test_data['Error'] ** 2

print('MSE (testing) = ' +str(np.mean(test_data['Error_squared'])))

In [ ]:
test_data

In [ ]:
# instead of computing MSE, can also compute R^2

X_test = test_data[['LoudnessdB']]
y_test = test_data['Energy']

print('R^2 score (testing) = ' + str(lin_model.score( X_test , y_test)))

### 2. Assessing the goodness of a regression model

$R^2$ score is the square of the correlation coefficient $R$.

Recall that MSE is the mean of the squares of the error, where error is measured by distance from the best-fit line.

$$R^2 = \dfrac{Error(mean) - Error(line)}{Error(mean)}$$
where 
- $Error(mean)$ is the sum of the errors between the data points and the mean of the data points and 
- $Error(line)$ is the sum of the errors between the data points and the line of the data points.

In our example, $R^2=0.39$. This means that there is 39% less variation around the line than the mean. Meaning, the `LoudnessdB` and `Energy` account for 39% of the variation. This means that 39% of the variation in the data is explained by the `LoudnessdB` and `Energy` relationship, something else must explain the remaining 61%.

In general, $R^2$ is the percentage of variation explained by the relationship between two variables.

### Randomly choosing rows for the training and test data

#### Useful new functions

- `np.arange( A, B)`
    - Creates a numpy array consisting of numbers starting from A, up to B (not including B)
- `np.random.choice( ARRAY, N , replace = False)`
    - Randomly selects N elements from ARRAY without replacement
- `np.setdiff1d( ARRAY1, ARRAY2)`
    - Returns an array containing unique values in ARRAY1 that are not in ARRAY2


**Training and Test Data Method 2:** Another way to split the data is to randomly choose half of the data set as the training and use the remaining half as the test data.

In [ ]:
# set up an array of all of the row indices
row_indices = np.arange(0, 50) # list of numbers from 0 to 49

# randomly select some row indices for the training data
training_row_indices = np.random.choice( row_indices , 25 , replace = False )

# the rest of the row indices are for the test data:
test_row_indices = np.setdiff1d( row_indices , training_row_indices  )

# pick out the rows of the big dataset based on the chosen row indices
training_data = top50.iloc[  training_row_indices  , : ]
test_data = top50.iloc[ test_row_indices , : ]

In [ ]:
training_data.shape

In [ ]:
test_data.shape

### Regression: What about fitting other curves?

Same idea as fitting a line: Find an equation for the curve, such that the MSE  (or other measure of error) is as small as possible.

**Example:** 

Suppose we want to fit a quadratic function $y = ax^2 + bx + c$ through data  points.

Suppose we have three data points $(x_1, y_1), (x_2, y_2), (x_3, y_3)$ that has the pattern of a parabola. Then we want these data points to satisfy the quadratic function above. The goal to is solve for the constants $a, b, c$ such that 
$$y_1 = ax_1^2 + bx_1 + c$$
$$y_2 = ax_2^2 + bx_2 + c$$
$$y_3 = ax_3^2 + bx_3 + c$$

In general, this system of equation has no solution, so we will find the "best-fit" quadratic function.

Idea: Find $a, b, c$ so that the MSE between the curve and the data points are  as small as possible.

For example, if one of the data points is $(2, 3)$, then we plug that into the quadratic equation:
$$3 = a(2^2) + b (2) + c \implies 3=4a+2b+c$$

Then $[4, 2, 1]$ will be one of the inputs in X, the training data. ie. one of the rows in our training data.

### Fitting a polynomial curve using linear regression via python

#### Plotting the best-fit polynomials

Plotting the polynomial curve (i.e., one that minimizes MSE):

    import seaborn as sns
    sns.regplot(data = DATAFRAMENAME, x = ‘COLNAME1’, y = ‘COLNAME2’, order = NUM)



In [ ]:
# plot
sns.regplot(  data = training_data , x = 'LoudnessdB', y = 'Energy', order = 3  )

#### Finding the equation of the best-fit polynomials

    from sklearn.linear_model import LinearRegression
    MODELNAME = LinearRegression().fit( X, Y )

Where
- X = a **dataframe** with the columns of the **independent variable**
- Y = a **list** containing the values of the **dependent variable**

And the columns of X includes powers of the variable of interest


In [ ]:
# find the equation of the cubic polynomial 

# energy = a * loudness^3 + b * loudness^2 + c * loudness + d
# linear regression: find coefficients a, b, c, d that minimizes the MSE

from sklearn.linear_model import LinearRegression

X_train = training_data[['LoudnessdB']]
y_train = training_data['Energy']


#add loudness squared and cubed columns in the X_train data frame
X_train['LoudnessdB_squared'] = X_train['LoudnessdB'] ** 2
X_train['LoudnessdB_cubed'] = X_train['LoudnessdB'] ** 3
X_train.head()

lin_model = LinearRegression().fit( X_train, y_train  )

m = lin_model.coef_
y_intercept = lin_model.intercept_

# predicted energy = a * loudness^3 + b * loudness^2 + c * loudness + d
print(m)  # the list of a , b, c
print(y_intercept) # the y intercept, aka d in the above equation in the comment

print(f"Equation of best cubic curve is y = {round(m[0], 3)}x^3 + {round(m[1],3)}x^2 + {round(m[2],3)}x + {round(y_intercept,3)}")

In [ ]:
X_train.head()

In [ ]:
# linear regression, with multiple (independent) variables 

from sklearn.linear_model import LinearRegression

X_train = training_data[['LoudnessdB', 'BeatsPerMinute']]
y_train = training_data['Energy']


lin_model_multiplevars = LinearRegression().fit( X_train, y_train  )

m = lin_model_multiplevars.coef_
z_intercept = lin_model_multiplevars.intercept_

# energy = a * loudnessdb + b * BeatsPerMinute + c
print(m) # the list of a , b
print(z_intercept) # the z intercept, aka c in the above equation in the comment

print(f"Equation of best linear curve is z = {round(m[0], 3)}x + {round(m[1],3)}y + {round(z_intercept,3)}")

**Training and Test Data Method 3:** Another useful way to split the data, is to use a built-in function in scikit-learn called `train_test_split`.

    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = FRACTION, random_state = NUMBER)


Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
from sklearn.model_selection import train_test_split

X = top50[['LoudnessdB', 'BeatsPerMinute']]
Y = top50['Energy']

# Be aware of the order of the items returned
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.5, random_state = 11)


In [ ]:
X_train.head()

In [ ]:
X_test.head()

## 3. Overfitting, Underfitting

### The importance of assessing models
(and of using different datasets for building vs. assessing models)

**Overfitting** is the problem that arises when
- the model fits the training data really well
- but does not make good predictions on new datasets.


**Overfitting Example:**

<table><tr>
    <td><img src='overfit.png' width=500></td>
    <td><img src='line.png' width=500></td></tr>
</table>

**Underfitting** is the problem that arises when
- the model is “too simple” and does not sufficiently capture patterns that arise in the data

**Underfitting Example:**

<img src='underfit.png' width=500>

**MSE** = Bias + Variance
- **Overfitting**: Model is too complex →  Low bias, high variance
- **Underfitting**: Model is too simple → High bias, low variance

